In [1]:
file_info <- read.table("/home/ylee/oRNAment/PWMs/conv.sh", header = FALSE, sep = " ")

In [2]:
file_info <- file_info[, -1]
colnames(file_info) <- c("base_name", "num_name")
head(file_info)

,base_name,num_name
,<chr>,<chr>
1,A1CF_7mer_logo0.PWM,001.PWM;
2,A1CF_7mer_logo1.PWM,002.PWM;
3,A1CF_7mer_logo2.PWM,003.PWM;
4,A1CF_7mer_logo3.PWM,004.PWM;
5,A1CF_7mer_logo4.PWM,005.PWM;
6,A1CF_7mer_logoM001.PWM,006.PWM;


In [3]:
file_info$num_name <- sub(";$", "", file_info$num_name)
file_info$RBP_name <- sub("_7mer.*", "", file_info$base_name)
file_info$motif_id <- gsub(".*logo(.*)\\.PWM", "\\1", file_info$base_name)
head(file_info)

,base_name,num_name,RBP_name,motif_id
,<chr>,<chr>,<chr>,<chr>
1,A1CF_7mer_logo0.PWM,001.PWM,A1CF,0
2,A1CF_7mer_logo1.PWM,002.PWM,A1CF,1
3,A1CF_7mer_logo2.PWM,003.PWM,A1CF,2
4,A1CF_7mer_logo3.PWM,004.PWM,A1CF,3
5,A1CF_7mer_logo4.PWM,005.PWM,A1CF,4
6,A1CF_7mer_logoM001.PWM,006.PWM,A1CF,M001


In [5]:
library(AnnotationDbi)
library(org.Hs.eg.db) # Human gene database
library(org.Mm.eg.db) # Mouse gene database

In [6]:
# bring valid key list
valid_human_keys <- c(keys(org.Hs.eg.db, keytype = "SYMBOL"), keys(org.Hs.eg.db, keytype = "ENSEMBL"))
valid_mouse_keys <- c(keys(org.Mm.eg.db, keytype = "SYMBOL"), keys(org.Mm.eg.db, keytype = "ENSEMBL"))

# check if the RBP_name is valid
file_info$valid_human <- file_info$RBP_name %in% valid_human_keys
file_info$valid_mouse <- file_info$RBP_name %in% valid_mouse_keys

head(file_info)

,base_name,num_name,RBP_name,motif_id,valid_human,valid_mouse
,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>
1,A1CF_7mer_logo0.PWM,001.PWM,A1CF,0,TRUE,FALSE
2,A1CF_7mer_logo1.PWM,002.PWM,A1CF,1,TRUE,FALSE
3,A1CF_7mer_logo2.PWM,003.PWM,A1CF,2,TRUE,FALSE
4,A1CF_7mer_logo3.PWM,004.PWM,A1CF,3,TRUE,FALSE
5,A1CF_7mer_logo4.PWM,005.PWM,A1CF,4,TRUE,FALSE
6,A1CF_7mer_logoM001.PWM,006.PWM,A1CF,M001,TRUE,FALSE


In [7]:
input_dir = "/home/ylee/oRNAment/PWMs/"
output_dir = "/home/ylee/oRNAment/oRNAment_PFM_species_filtered/"

for (i in 1:nrow(file_info)) {
    RBP <- file_info[i, "RBP_name"]
    
    if (file_info[i, "valid_human"] || file_info[i, "valid_mouse"]) {
        input_file <- file.path(paste0(input_dir, file_info[i, "num_name"]))
        output_file <- file.path(paste0(output_dir, file_info[i, "RBP_name"], "_PWM_", file_info[i, "motif_id"], ".txt"))
        
        motif_data <- read.table(input_file, header = TRUE, sep = "\t", stringsAsFactors = FALSE)
        matrix_data <- motif_data[, -1]  # Remove the first column ("P0")    
        
        write.table(matrix_data, output_file, sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
        print(paste("Saved:", output_file))
    }
}

[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/A1CF_PWM_0.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/A1CF_PWM_1.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/A1CF_PWM_2.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/A1CF_PWM_3.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/A1CF_PWM_4.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/A1CF_PWM_M001.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/ANKHD1_PWM_M002.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/BOLL_PWM_0.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/BOLL_PWM_1.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/BOLL_PWM_2.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/BOLL_PWM_3.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_filtered/CELF1_PWM_0.txt"
[1] "Saved: /home/ylee/oRNAment/oRNAment_PFM_species_fi

## Summary_original_info
##### Summary that includes the original database information

In [18]:
base_dir <- "/home/ylee/oRNAment/oRNAment_meme"

In [20]:
files <- list.files(base_dir, full.names = TRUE)

file_info <- data.frame(
    full_path = files,
    base_name = basename(files),
    stringsAsFactors = FALSE
)

file_info$RBP_name <- sub("_PWM.*", "", file_info$base_name)
file_info$motif_id <- sub(".meme", "", sub(".*PWM_", "", file_info$base_name))

In [21]:
for (i in 1:nrow(file_info)) {
    lines <- readLines(file_info[i, "full_path"])
    motif_line <- grep("^MOTIF", lines, value = TRUE)  # Find line starting with "MOTIF"
    motif_name <- sub("^MOTIF \\d+\\s+", "", motif_line)  # Remove "MOTIF <number> " to get the name
    file_info[i, "motif_alt_id"] <- motif_name
}

In [22]:
head(file_info)

,full_path,base_name,RBP_name,motif_id,motif_alt_id
,<chr>,<chr>,<chr>,<chr>,<chr>
1,/home/ylee/oRNAment/oRNAment_meme/A1CF_PWM_0.meme,A1CF_PWM_0.meme,A1CF,0,NTAATTA
2,/home/ylee/oRNAment/oRNAment_meme/A1CF_PWM_1.meme,A1CF_PWM_1.meme,A1CF,1,VAATCAN
3,/home/ylee/oRNAment/oRNAment_meme/A1CF_PWM_2.meme,A1CF_PWM_2.meme,A1CF,2,SGGRCTG
4,/home/ylee/oRNAment/oRNAment_meme/A1CF_PWM_3.meme,A1CF_PWM_3.meme,A1CF,3,AGHTCGG
5,/home/ylee/oRNAment/oRNAment_meme/A1CF_PWM_4.meme,A1CF_PWM_4.meme,A1CF,4,AATAWAV
6,/home/ylee/oRNAment/oRNAment_meme/A1CF_PWM_M001.meme,A1CF_PWM_M001.meme,A1CF,M001,WTAATTR


In [23]:
file_info <- file_info[, -c(1, 2)]
head(file_info)

,RBP_name,motif_id,motif_alt_id
,<chr>,<chr>,<chr>
1,A1CF,0,NTAATTA
2,A1CF,1,VAATCAN
3,A1CF,2,SGGRCTG
4,A1CF,3,AGHTCGG
5,A1CF,4,AATAWAV
6,A1CF,M001,WTAATTR


In [24]:
length(unique(file_info$RBP_name))

[1] 125

In [25]:
nrow(file_info)

[1] 323

In [26]:
write.csv(file_info, "/home/ylee/oRNAment/oRNAment_RBP_motifs_original_db_info.csv", row.names = FALSE)

## Summary
##### Summary that includes annotation of fimo

In [53]:
library(dplyr)
library(tidyr)

In [54]:
base_dir <- "/home/ylee/oRNAment/oRNAment_meme_combined"

In [55]:
files <- list.files(base_dir, full.names = TRUE)
rbp_list <- sub("_combined.*", "", basename(files))
length(rbp_list)

[1] 125

In [56]:
# Extract RBP related motifs from the MEME file
extract_motifs <- function(rbp_list, database) { 
    RBP_motifs <- data.frame(
        RBP_name = character(),
        motif_id = character(),
        motif_alt_id = character()    
    )
    
    for (rbp in rbp_list){
        RBP_name <- rbp
        motif_id <- NULL
        motif_alt_id <- NULL
        
        meme_file <- paste0(base_dir, "/", rbp, "_combined_", database, ".meme")
        lines <- readLines(meme_file)

        for (line in lines) {
            if (grepl("^MOTIF", line)) {
              motif_id <- sub("MOTIF (\\S+) (\\S+)", "\\1", line)  # extract motif ID
              motif_alt_id <- sub("MOTIF (\\S+) (\\S+)", "\\2", line)  # extract motif alternative ID
            }
            if (grepl("^letter-probability matrix", line)) {
                new_row <- data.frame(
                    RBP_name = rbp,
                    motif_id = motif_id,
                    motif_alt_id = motif_alt_id
                )
                RBP_motifs <- bind_rows(RBP_motifs, new_row)
            }
        }
    }
  
  return(RBP_motifs)
}

In [57]:
RBP_motifs <- extract_motifs(rbp_list, "oRNAment")
head(RBP_motifs)

,RBP_name,motif_id,motif_alt_id
,<chr>,<chr>,<chr>
1,A1CF,1.1,NTAATTA
2,A1CF,1.2,VAATCAN
3,A1CF,1.3,SGGRCTG
4,A1CF,1.4,AGHTCGG
5,A1CF,1.5,AATAWAV
6,A1CF,1.6,WTAATTR


In [58]:
length(unique(RBP_motifs$RBP_name))
nrow(RBP_motifs)

[1] 125

[1] 323

In [59]:
write.csv(RBP_motifs, "/home/ylee/oRNAment/oRNAment_RBP_motifs.csv", row.names = FALSE)